In [337]:
# Installations of libraries
!pip install PyPDF2
!pip install nltk
!pip install pdfminer
!pip install textblob

In [326]:
# nltk download
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [301]:
# Imports
from PyPDF2 import PdfFileReader, PdfFileWriter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from textblob import TextBlob

import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

import io
import nltk

In [302]:
# Initiate pdf file
pdf_file = "../res/who_int_0.pdf"
# Initiate dump file
temp_text = "../dump/temp.txt"

In [399]:
# https://stackoverflow.com/questions/64467930/python-pypdf2-extracttext-not-working
inFile = open(pdf_file, 'rb')
resMgr = PDFResourceManager()
retData = io.StringIO()
TxtConverter = TextConverter(resMgr, retData,laparams = LAParams())
interpreter = PDFPageInterpreter(resMgr, TxtConverter)

# Process each pages
for page in PDFPage.get_pages(inFile): 
    interpreter.process_page(page)
    

# Write to text temporary file
txt = retData.getvalue()

In [409]:
# Acquire sentences
blob = TextBlob(txt)
sentences = []

for s in blob.sentences:
     sentences.append(str(s).strip()),

cleaned_sentences = []
for x in sentences:
     cleaned_sentences.append(x.replace("\n", " "))

cleaned_sentences[0:2]

['3/15/22, 10:32 AM  Is the pandemic ending soon?',
 'Western Pacific Philippines  ©  Opinion: Is the pandemic ending soon?']

In [415]:
with open(temp_text, 'w') as f: 
     f.write('\n'.join(cleaned_sentences))

In [416]:
raw = open(outTXTFile).read()

In [417]:
# Length and Type of raw text file
print(len(raw), type(raw))

7816 <class 'str'>


In [430]:
sentences = []

with open(outTXTFile) as file:
    while line := file.readline():
        sentences.append(line.rstrip())

In [453]:
sentence_index = []
tokens = []

for idx1 in range(len(sentences)):
    temporary_sentence = sentences[idx1]
    temporary_tokens = nltk.word_tokenize(temporary_sentence)
    
    for idx2, val in enumerate(temporary_tokens):
        sentence_index.append(idx1)
        tokens.append(val)

sentence_index

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,


In [454]:
# Length must be the same as the original tokens.
print(len(sentence_index))

1353


In [455]:
# Length of tokenized text file
print(len(tokens), type(tokens))

1353 <class 'list'>


In [456]:
print(tokens[0:69])

['3/15/22', ',', '10:32', 'AM', 'Is', 'the', 'pandemic', 'ending', 'soon', '?', 'Western', 'Pacific', 'Philippines', '©', 'Opinion', ':', 'Is', 'the', 'pandemic', 'ending', 'soon', '?', '11', 'March', '2022', 'Statement', '|', 'By', 'Dr', 'Rajendra', 'Yadav', 'and', 'Dr', 'Sangjun', 'Moon', ',', 'WHO', 'Philippines', 'Manila', ',', 'Philippines', '(', '11', 'March', '2022', ')', '–', 'When', 'will', 'this', 'end', '?', 'This', 'question', 'has', 'been', 'on', 'everyone', '’', 's', 'mind', 'for', 'many', 'months', '.', 'We', 'are', 'all', 'tired']


In [457]:
it = iter(tokens)
size = [i for i in range(len(tokens))]

pos_tags = nltk.pos_tag(tokens)
words, tags = zip(*nltk.pos_tag(tokens))

In [465]:
df = pd.DataFrame(list(zip(size, sentence_index, words, tags)), columns=["Token_ID", "Sentence_Index", "Token", "Pos_Tags"])
display(df)

,Token_ID,Sentence_Index,Token,Pos_Tags
0,0,0,3/15/22,CD
1,1,0,",",","
2,2,0,10:32,CD
3,3,0,AM,NNP
4,4,0,Is,VBZ
...,...,...,...,...
1348,1348,72,.,.
1349,1349,73,https,NN
1350,1350,73,:,:
1351,1351,73,//www.who.int/philippines/news/detail/11-03-20...,JJ


In [466]:
df.to_csv ('../data/csv_dataset/data.csv', index=False, header=True)

In [467]:
chunks = nltk.ne_chunk(pos_tags, binary=True)
for chunk in chunks:
    print(chunk)

('3/15/22', 'CD')
(',', ',')
('10:32', 'CD')
('AM', 'NNP')
('Is', 'VBZ')
('the', 'DT')
('pandemic', 'JJ')
('ending', 'VBG')
('soon', 'RB')
('?', '.')
(NE Western/JJ Pacific/NNP Philippines/NNP)
('©', 'NNP')
('Opinion', 'NNP')
(':', ':')
('Is', 'VBZ')
('the', 'DT')
('pandemic', 'JJ')
('ending', 'VBG')
('soon', 'RB')
('?', '.')
('11', 'CD')
('March', 'NNP')
('2022', 'CD')
('Statement', 'NNP')
('|', 'NN')
('By', 'IN')
('Dr', 'NNP')
('Rajendra', 'NNP')
('Yadav', 'NNP')
('and', 'CC')
('Dr', 'NNP')
('Sangjun', 'NNP')
('Moon', 'NNP')
(',', ',')
(NE WHO/NNP Philippines/NNPS Manila/NNP)
(',', ',')
(NE Philippines/NNP)
('(', '(')
('11', 'CD')
('March', 'NNP')
('2022', 'CD')
(')', ')')
('–', 'NN')
('When', 'WRB')
('will', 'MD')
('this', 'DT')
('end', 'NN')
('?', '.')
('This', 'DT')
('question', 'NN')
('has', 'VBZ')
('been', 'VBN')
('on', 'IN')
('everyone', 'NN')
('’', 'JJ')
('s', 'NN')
('mind', 'NN')
('for', 'IN')
('many', 'JJ')
('months', 'NNS')
('.', '.')
('We', 'PRP')
('are', 'VBP')
('all', 'D

In [468]:
entities = []
labels = []

for chunk in chunks:
    if hasattr(chunk, 'label'):
        entities.append(' '.join(c[0] for c in chunk))
        labels.append(chunk.label())
        
entities_labels = list(set(zip(entities, labels)))
entities_labels_df = pd.DataFrame(entities_labels)
entities_labels_df.columns = ['Entities', 'Labels']
entities_labels

[('COVAX Facility', 'NE'),
 ('Western Pacific Philippines', 'NE'),
 ('Universal Health Care', 'NE'),
 ('Epidemic Intelligence', 'NE'),
 ('Tedros Ghebreyesus', 'NE'),
 ('Delta', 'NE'),
 ('WHO Philippines Manila', 'NE'),
 ('WHO Philippines', 'NE'),
 ('National Vaccination Operations Center', 'NE'),
 ('Pandemic', 'NE'),
 ('WHO', 'NE'),
 ('UHC', 'NE'),
 ('Omicron', 'NE'),
 ('Philippines', 'NE')]